In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/home/jatin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jatin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jatin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
df = pd.read_csv('train.csv')

In [7]:
df = df[['Survived','Pclass','Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [8]:
df = df.dropna(axis=0, how='any')

In [9]:
y = df['Survived']

In [10]:
X = df[['Pclass','Sex', 'Age', 'SibSp', 'Parch','Fare', 'Embarked']]

In [11]:
from sklearn.preprocessing import LabelEncoder
X = np.array(X)

In [12]:
le_1 = LabelEncoder()

In [13]:
X[:, 1] = le_1.fit_transform(X[:, 1])

In [14]:
le_2 = LabelEncoder()
X[:, 6] = le_2.fit_transform(X[:, 6])

In [15]:
X = pd.DataFrame(data=X, columns=['Pclass','Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])

In [16]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
sc = StandardScaler()

In [19]:
sc.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

## Evaluation Function 

In [21]:
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [22]:
def print_score(clf, X_train, X_test, y_train, y_test, train = True):
#     print accuracy score, classification report, confusion metrics
    if train:
#         training performance
        print('Train Result : \n')
        print('Accuracy Score {0:.4f}\n'.format(accuracy_score(y_train, clf.predict(X_train))))
        print('Classification Report : \n {} \n'.format(classification_report(y_train, clf.predict(X_train))))
        print('Confusion Metrics : \n {} \n'.format(confusion_matrix(y_train, clf.predict(X_train))))
        
        res = cross_val_score(clf, X_train, y_train, cv = 10, scoring='accuracy')
        print('Average Accuracy : {0:.4f}\n'.format(np.mean(res)))
        print('Accuracy SD : {0:.4f}\n'.format(np.std(res)))
        
    elif train == False:
#         test performance
        print('Test Result : \n')
        print('Accuracy Score {0:.4f}\n'.format(accuracy_score(y_test, clf.predict(X_test))))
        print('Classification Report : \n {}\n'.format(classification_report(y_test, clf.predict(X_test))))
        print('Confusion Metrics : \n {} \n'.format(confusion_matrix(y_test, clf.predict(X_test))))


## SVM

In [23]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [24]:
clf = RandomForestClassifier(criterion='entropy', n_estimators=200, max_depth=5, min_samples_split=20)

In [25]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
print_score(clf, X_train, X_test, y_train, y_test, train=True)
print_score(clf, X_train, X_test, y_train, y_test, train=False)

Train Result : 

Accuracy Score 0.8541

Classification Report : 
              precision    recall  f1-score   support

          0       0.82      0.97      0.89       344
          1       0.93      0.68      0.79       225

avg / total       0.86      0.85      0.85       569
 

Confusion Metrics : 
 [[332  12]
 [ 71 154]] 

Average Accuracy : 0.8068

Accuracy SD : 0.0331

Test Result : 

Accuracy Score 0.8042

Classification Report : 
              precision    recall  f1-score   support

          0       0.77      0.93      0.84        80
          1       0.87      0.65      0.75        63

avg / total       0.82      0.80      0.80       143


Confusion Metrics : 
 [[74  6]
 [22 41]] 



## Using grid search

In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
params = {'criterion': ['entropy', 'gini'],
          'n_estimators': [100, 200, 300, 400],
          'min_samples_split': [10, 20, 30, 40,50],
          'max_depth': [3, 4, 5, 6, 7, 8]}

In [29]:
grid_cv = GridSearchCV(clf, param_grid=params, n_jobs=-1)

In [30]:
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['entropy', 'gini'], 'n_estimators': [100, 200, 300, 400], 'min_samples_split': [10, 20, 30, 40, 50], 'max_depth': [3, 4, 5, 6, 7, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
print_score(grid_cv, X_train, X_test, y_train, y_test, train=True)
print_score(grid_cv, X_train, X_test, y_train, y_test, train=False)

Train Result : 

Accuracy Score 0.8981

Classification Report : 
              precision    recall  f1-score   support

          0       0.88      0.96      0.92       344
          1       0.93      0.80      0.86       225

avg / total       0.90      0.90      0.90       569
 

Confusion Metrics : 
 [[331  13]
 [ 45 180]] 

Average Accuracy : 0.8103

Accuracy SD : 0.0413

Test Result : 

Accuracy Score 0.7832

Classification Report : 
              precision    recall  f1-score   support

          0       0.76      0.89      0.82        80
          1       0.82      0.65      0.73        63

avg / total       0.79      0.78      0.78       143


Confusion Metrics : 
 [[71  9]
 [22 41]] 



In [32]:
grid_cv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)